# Seizure Detection Using LFP and AUX Data: Feature Extraction, Windowing, and Modeling

## Initial Setup

### Import Libaries

In [1]:
import pandas as pd
import numpy as np
from scipy.signal import welch, get_window
import h5py

### Load EEG Data

In [2]:
# Path to the .mat file
mat_file_path = '/Users/hyojin/Desktop/Dr.Kim/2000Hz/POST_KA_B_F1_2024-10-03_13-06-07_2000Hz.mat'

# Load .mat file using h5py for MATLAB v7.3 format
with h5py.File(mat_file_path, 'r') as mat_file:
    eeg_data = np.array(mat_file['eegData']).T  # Transpose to get shape (channels, time)
    print(f"LFP data shape: {eeg_data.shape}")

LFP data shape: (32, 7214080)


### Set Parameters

In [3]:
# Sampling frequency
fs = 2000  # Brainstorm sampling rate in Hz

# Define frequency bands (Brainstorm custom bands)
bands = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'slowgamma': (30, 50),
    'fastgamma': (50, 100)
}

# Define PSD computation parameters
window_length = 2 * fs  # 2-second window
noverlap = window_length // 2  # 50% overlap
window = get_window('hamming', window_length)  # Hamming window


## PSD Computation for All Channels

### Initialize Storage for PSD Results

In [4]:
# Initialize dictionary to store band PSD values for all channels
all_band_psd = {band: [] for band in bands}

### PSD by Channels

In [5]:
# Process each channel
for channel_idx, channel_data in enumerate(eeg_data):
    print(f"Processing channel {channel_idx + 1}/{eeg_data.shape[0]}...")

    # Calculate PSD for the channel using Welch's method
    f, psd = welch(channel_data, fs, window=window, nperseg=window_length, noverlap=noverlap, scaling='density')

    # Convert PSD to V²/Hz
    psd_in_v2 = psd / 1e12  # Convert from μV²/Hz to V²/Hz

    # Calculate mean PSD values for each frequency band
    for band_name, (f_low, f_high) in bands.items():
        band_indices = (f >= f_low) & (f <= f_high)
        band_mean_psd = np.mean(psd_in_v2[band_indices])  # Mean power in the band
        all_band_psd[band_name].append(band_mean_psd)


Processing channel 1/32...
Processing channel 2/32...
Processing channel 3/32...
Processing channel 4/32...
Processing channel 5/32...
Processing channel 6/32...
Processing channel 7/32...
Processing channel 8/32...
Processing channel 9/32...
Processing channel 10/32...
Processing channel 11/32...
Processing channel 12/32...
Processing channel 13/32...
Processing channel 14/32...
Processing channel 15/32...
Processing channel 16/32...
Processing channel 17/32...
Processing channel 18/32...
Processing channel 19/32...
Processing channel 20/32...
Processing channel 21/32...
Processing channel 22/32...
Processing channel 23/32...
Processing channel 24/32...
Processing channel 25/32...
Processing channel 26/32...
Processing channel 27/32...
Processing channel 28/32...
Processing channel 29/32...
Processing channel 30/32...
Processing channel 31/32...
Processing channel 32/32...


### Create and Save Results DataFrame

In [6]:
# Create a DataFrame from the results
df_psd = pd.DataFrame(all_band_psd)
df_psd.index = [f'Channel {i + 1}' for i in range(eeg_data.shape[0])]

# Display the DataFrame
print("\nPSD DataFrame (in V²/Hz):")
print(df_psd)

# Optionally save the DataFrame to a CSV file
output_csv_path = '/Users/hyojin/Desktop/Dr.Kim/Dataframe/psd_results.csv'
df_psd.to_csv(output_csv_path, index=True)
print(f"PSD results saved to {output_csv_path}")



PSD DataFrame (in V²/Hz):
                   delta         theta         alpha          beta  \
Channel 1   8.053666e-09  3.453589e-09  1.310373e-09  3.864253e-10   
Channel 2   8.889110e-09  3.427332e-09  1.253667e-09  3.726808e-10   
Channel 3   9.147446e-09  3.401054e-09  1.024884e-09  2.980524e-10   
Channel 4   9.244176e-09  3.215013e-09  8.250956e-10  2.850025e-10   
Channel 5   1.126136e-08  3.508617e-09  7.841206e-10  3.005138e-10   
Channel 6   1.278625e-08  3.744832e-09  7.824130e-10  2.904076e-10   
Channel 7   1.455558e-08  4.263450e-09  8.693633e-10  2.819331e-10   
Channel 8   1.376630e-08  4.143737e-09  8.553016e-10  2.810650e-10   
Channel 9   1.239517e-08  3.775058e-09  8.096397e-10  2.726325e-10   
Channel 10  9.131103e-09  2.878737e-09  6.810299e-10  2.377403e-10   
Channel 11  6.998429e-09  2.285200e-09  6.043004e-10  2.145141e-10   
Channel 12  3.075219e-09  1.261505e-09  3.821750e-10  1.354794e-10   
Channel 13  3.119857e-09  1.263087e-09  3.933471e-10  1.407978e

## PSD by 2s window with 50% overlap in one channel(in CA1 area)

### Select and Process Channel 14

In [7]:
# Select channel 14 (Python is 0-indexed, so index 13)
channel_idx = 13
channel_data = eeg_data[channel_idx, :]

# Calculate PSD for the channel using Welch's method
f, psd = welch(
    channel_data,
    fs,
    window=window,
    nperseg=window_length,
    noverlap=noverlap,
    scaling='density',
    return_onesided=True,
)

# Convert PSD to V²/Hz (assuming input data is in μV²/Hz)
psd_in_v2 = psd / 1e12  # Convert from μV²/Hz to V²/Hz


### Compute PSD for 2-Second Windows

In [8]:
# Split the signal into windows and compute PSD for each window
n_samples = len(channel_data)
step_size = window_length // 2
start_indices = np.arange(0, n_samples - window_length + 1, step_size)

# Store results in a DataFrame
rows = []
for start in start_indices:
    segment = channel_data[start:start + window_length]
    f, psd = welch(
        segment,
        fs,
        window=window,
        nperseg=window_length,
        noverlap=0,  # No additional overlap for this small segment
        scaling='density',
        return_onesided=True,
    )
    psd_in_v2 = psd / 1e12  # Convert from μV²/Hz to V²/Hz

    # Calculate mean PSD values for each frequency band
    row = {}
    for band_name, (f_low, f_high) in bands.items():
        band_indices = (f >= f_low) & (f <= f_high)
        row[band_name] = np.mean(psd_in_v2[band_indices])  # Mean power in the band
    rows.append(row)


### Create and Save Results DataFrame

In [9]:
# Create a DataFrame for the PSD results
df_psd_windows = pd.DataFrame(rows)
df_psd_windows.index = [f"{i}-{i+2}s" for i in range(len(rows))]

# Display the DataFrame
print("\nPSD DataFrame for Channel 14 (in V²/Hz):")
print(df_psd_windows)

# Optionally save the DataFrame to a CSV file
output_csv_path = '/Users/hyojin/Desktop/Dr.Kim/Dataframe/psd_channel_14_results.csv'
df_psd_windows.to_csv(output_csv_path, index=True)
print(f"PSD results saved to {output_csv_path}")



PSD DataFrame for Channel 14 (in V²/Hz):
                   delta         theta         alpha          beta  \
0-2s        1.605210e-11  1.825020e-11  7.879450e-12  2.768594e-12   
1-3s        1.420186e-11  2.014871e-11  2.771339e-12  7.045722e-12   
2-4s        6.840039e-12  1.248694e-11  3.861962e-12  6.393010e-12   
3-5s        1.681903e-11  3.516268e-11  9.588216e-12  6.385192e-12   
4-6s        8.960209e-12  3.708496e-11  6.885384e-12  3.597341e-12   
...                  ...           ...           ...           ...   
3601-3603s  2.125099e-09  5.937623e-10  4.833021e-10  1.545539e-10   
3602-3604s  1.897520e-09  4.941052e-10  4.563004e-11  1.409748e-11   
3603-3605s  2.489903e-09  5.034880e-10  6.528881e-11  1.174057e-11   
3604-3606s  2.525036e-09  2.921870e-10  2.836990e-10  3.678167e-11   
3605-3607s  3.154934e-09  6.919436e-10  3.378061e-11  5.481980e-12   

               slowgamma     fastgamma  
0-2s        9.102647e-12  1.915070e-12  
1-3s        8.364090e-12  2.029099e

## LFP Feature Extraction: Signal Energy and RMS

In [10]:
# Compute additional features: Signal Energy and RMS
lfp_features = []

# Function to compute RMS
def compute_rms(signal):
    return np.sqrt(np.mean(signal**2))

# Process each channel to compute additional features
for channel_idx, channel_data in enumerate(eeg_data):
    print(f"Processing additional features for Channel {channel_idx + 1}...")

    # Compute signal energy
    signal_energy = np.sum(channel_data**2) / len(channel_data)

    # Compute RMS
    rms_value = compute_rms(channel_data)

    # Append features to the list
    lfp_features.append({
        'channel': f'Channel {channel_idx + 1}',
        'signal_energy': signal_energy,
        'rms': rms_value
    })

# Convert additional features to a DataFrame
df_additional_features = pd.DataFrame(lfp_features)
df_additional_features.set_index('channel', inplace=True)

# Combine with existing PSD DataFrame
df_combined_features = pd.concat([df_psd, df_additional_features], axis=1)

# Display combined features DataFrame
print("\nCombined LFP Features DataFrame:")
print(df_combined_features)

# Optionally save combined features to a CSV
output_csv_path_combined = '/Users/hyojin/Desktop/Dr.Kim/Dataframe/combined_lfp_features.csv'
df_combined_features.to_csv(output_csv_path_combined, index=True)
print(f"Combined LFP features saved to {output_csv_path_combined}")


Processing additional features for Channel 1...
Processing additional features for Channel 2...
Processing additional features for Channel 3...
Processing additional features for Channel 4...
Processing additional features for Channel 5...
Processing additional features for Channel 6...
Processing additional features for Channel 7...
Processing additional features for Channel 8...
Processing additional features for Channel 9...
Processing additional features for Channel 10...
Processing additional features for Channel 11...
Processing additional features for Channel 12...
Processing additional features for Channel 13...
Processing additional features for Channel 14...
Processing additional features for Channel 15...
Processing additional features for Channel 16...
Processing additional features for Channel 17...
Processing additional features for Channel 18...
Processing additional features for Channel 19...
Processing additional features for Channel 20...
Processing additional feature

## Label Seizure Windows

In [12]:
# Define new seizure intervals in seconds
seizure_intervals = [
    (52 * 60 + 12, 52 * 60 + 15),  # 52:12 - 52:15
    (52 * 60 + 26, 52 * 60 + 29),  # 52:26 - 52:29
    (52 * 60 + 54, 52 * 60 + 56),  # 52:54 - 52:56
    (54 * 60 + 32, 54 * 60 + 34),  # 54:32 - 54:34
    (58 * 60 + 16, 58 * 60 + 21)   # 58:16 - 58:21
]

# Convert intervals to sample indices
seizure_intervals_samples = [
    (int(start * fs), int(end * fs)) for start, end in seizure_intervals
]

def label_window(start_sample, end_sample, intervals):
    """
    Determine if a given window overlaps with any seizure interval.

    Parameters:
    - start_sample: Start index of the window
    - end_sample: End index of the window
    - intervals: List of (start, end) intervals in samples

    Returns:
    - 1 if the window overlaps with any seizure interval, otherwise 0
    """
    for interval_start, interval_end in intervals:
        if start_sample < interval_end and end_sample > interval_start:
            return 1  # Seizure
    return 0  # Non-seizure

# Label each window in your data
window_labels = []  # To store labels for each window
for start_idx in start_indices:  
    end_idx = start_idx + window_length  # End index of the current window
    label = label_window(start_idx, end_idx, seizure_intervals_samples)
    window_labels.append(label)

# Add labels to the PSD DataFrame
df_psd_windows['label'] = window_labels

# Save the labeled DataFrame
output_csv_path_labeled = '/Users/hyojin/Desktop/Dr.Kim/Dataframe/psd_channel_labeled.csv'
df_psd_windows.to_csv(output_csv_path_labeled, index=True)
print(f"Labeled PSD results saved to {output_csv_path_labeled}")

# Display a preview of the labeled DataFrame
print("\nLabeled PSD DataFrame:")
print(df_psd_windows.head())

Labeled PSD results saved to /Users/hyojin/Desktop/Dr.Kim/Dataframe/psd_channel_labeled.csv

Labeled PSD DataFrame:
             delta         theta         alpha          beta     slowgamma  \
0-2s  1.605210e-11  1.825020e-11  7.879450e-12  2.768594e-12  9.102647e-12   
1-3s  1.420186e-11  2.014871e-11  2.771339e-12  7.045722e-12  8.364090e-12   
2-4s  6.840039e-12  1.248694e-11  3.861962e-12  6.393010e-12  8.905623e-12   
3-5s  1.681903e-11  3.516268e-11  9.588216e-12  6.385192e-12  1.180134e-11   
4-6s  8.960209e-12  3.708496e-11  6.885384e-12  3.597341e-12  9.917462e-12   

         fastgamma  label  
0-2s  1.915070e-12      0  
1-3s  2.029099e-12      0  
2-4s  1.694824e-12      0  
3-5s  1.655262e-12      0  
4-6s  1.854515e-12      0  
